In [1]:
import openai
import os
openai.api_key = os.environ['OPENAI_API_KEY']


In [2]:
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import OpenAIWhisperParser
from langchain.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader

In [3]:
url="https://www.youtube.com/watch?v=jGwO_UgTS7I"
save_dir="docs/youtube/"
utube_loader = GenericLoader(YoutubeAudioLoader([url],save_dir),OpenAIWhisperParser())

In [4]:
utube_loader.load()

[youtube] Extracting URL: https://www.youtube.com/watch?v=jGwO_UgTS7I
[youtube] jGwO_UgTS7I: Downloading webpage
[youtube] jGwO_UgTS7I: Downloading ios player API JSON
[youtube] jGwO_UgTS7I: Downloading player 84314bef
[youtube] jGwO_UgTS7I: Downloading m3u8 information
[info] jGwO_UgTS7I: Downloading 1 format(s): 140
[download] Destination: docs\youtube\Stanford CS229： Machine Learning Course, Lecture 1 - Andrew Ng (Autumn 2018).m4a
[download] 100% of   69.76MiB in 00:00:02 at 31.09MiB/s    


ERROR: Postprocessing: ffprobe and ffmpeg not found. Please install or provide the path using --ffmpeg-location


DownloadError: ERROR: Postprocessing: ffprobe and ffmpeg not found. Please install or provide the path using --ffmpeg-location

In [5]:
from langchain.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
loader = WebBaseLoader("https://stackoverflow.co/teams/ai/?utm_medium=referral&utm_source=stackoverflow-community&utm_campaign=top-nav-bar&utm_content=overflowai")

In [7]:
documents = loader.load()

In [8]:
print(f'The number of pages from the document is {len(documents)}')
documents[0].page_content

The number of pages from the document is 1


'\n\nEmpower your organization for an AI future with Stack Overflow for Teams - Stack Overflow\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nProductsMenu Why Teams? Our solutionIntegrationsFeaturesCustomer SuccessSecurityReturn on Investment (ROI) Use cases  Engineers  Data Scientists  DevOps & SREs  Support  Product Management  Customers  Resources Learning centerCustomer help centerFrequently Asked Questions OverflowAI  Pricing  Log in  Try free  Contact us Stack Overflow for Teams Where developers and technologists share private knowledge with coworkers.Stack Overflow Advertising Where companies reach the world’s largest audience of developers and technologists.Join the community Where developers and technologists go to gain and share knowledge. GenAI features for Teams and our public platform  Train and fine-tune large language models Blog Writing on software a

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter, TokenTextSplitter

In [10]:
text2 = 'abcdefghijklmnopqrstuvwxyzabcdefg'
char_text_splitter = CharacterTextSplitter(chunk_size = 15, chunk_overlap = 4, separator = '')
re_char_text_splitter = RecursiveCharacterTextSplitter(chunk_size = 150, chunk_overlap = 4)


In [11]:
char_text_splitter.split_text(text2)

['abcdefghijklmno', 'lmnopqrstuvwxyz', 'wxyzabcdefg']

In [12]:
re_char_text_splitter.split_text(text2)

['abcdefghijklmnopqrstuvwxyzabcdefg']

In [13]:
token_splitter = TokenTextSplitter(chunk_size=8, chunk_overlap=2)
token_splitter.split_text(text2)

['abcdefghijklmnopq', 'opqrstuvwxyz', 'xyzabcdefg']

In [14]:
# each splitter has two functions split_text and split_document 
# the split_text function is used to split a str object into multiple strings
# the split_document function is used to split documents loaded by the document loader
# once the split_document function splits the document pages further, it sustains the metadata across all documents
# any splitter has few characterstics that define it: the chunk size
# the splitting logic, the chunk overlap and the logic to find the length of a split data: this length is used to determine the size of any chunk

In [15]:
split_pages = re_char_text_splitter.split_documents(documents)

In [16]:
print(f'The documents have been split into {len(split_pages)}')
split_pages[0]

The documents have been split into 57


Document(page_content='Empower your organization for an AI future with Stack Overflow for Teams - Stack Overflow', metadata={'source': 'https://stackoverflow.co/teams/ai/?utm_medium=referral&utm_source=stackoverflow-community&utm_campaign=top-nav-bar&utm_content=overflowai', 'title': 'Empower your organization for an AI future with Stack Overflow for Teams - Stack Overflow', 'description': 'Build a collective, enterprise-grade knowledge base around technology transformations, including GenAI and AI/ML.', 'language': 'en'})

In [17]:
from langchain.document_loaders import PyPDFLoader
cs229_loader = PyPDFLoader('https://cs229.stanford.edu/lectures-spring2022/main_notes.pdf')
cs229_docs = cs229_loader.load()
print(f'The number of documents loaded from cs229 are {len(cs229_docs)}')
cs229_docs[0]

The number of documents loaded from cs229 are 216


Document(page_content='CS229 Lecture Notes\nAndrew Ng\nUpdated by Tengyu Ma', metadata={'source': 'https://cs229.stanford.edu/lectures-spring2022/main_notes.pdf', 'page': 0})

In [18]:
rec_char_splitter = RecursiveCharacterTextSplitter(chunk_size = 1500, chunk_overlap = 150)
split_pages = rec_char_splitter.split_documents(cs229_docs)
print(f'The CS229 documents have been split into {len(split_pages)} pages')
split_pages[0]

The CS229 documents have been split into 352 pages


Document(page_content='CS229 Lecture Notes\nAndrew Ng\nUpdated by Tengyu Ma', metadata={'source': 'https://cs229.stanford.edu/lectures-spring2022/main_notes.pdf', 'page': 0})

In [19]:
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(api_key = api_key)
save_directory = '/chroma/persist1/'
vectordb = Chroma.from_documents(documents = split_pages, embedding= embedding,persist_directory=save_directory)
# we can persist the embeddings extracted within the memory with Chroma, instead of connecting to a external database server

In [20]:
vectordb._collection.count()

704

In [21]:
query = 'Who are the authors of CS229 lecture notes?'
vectordb.similarity_search_with_score(query,k=2)

[(Document(page_content='CS229 Lecture Notes\nAndrew Ng\nUpdated by Tengyu Ma', metadata={'page': 0, 'source': 'https://cs229.stanford.edu/lectures-spring2022/main_notes.pdf'}),
  0.27768057584762573),
 (Document(page_content='CS229 Lecture Notes\nAndrew Ng\nUpdated by Tengyu Ma', metadata={'page': 0, 'source': 'https://cs229.stanford.edu/lectures-spring2022/main_notes.pdf'}),
  0.27768057584762573)]

In [22]:
query = 'What does the lecture notes say about regression in page 20'
vectordb.similarity_search_with_score(query,k=3)

[(Document(page_content='Contents\nI Supervised learning 5\n1 Linear regression 8\n1.1 LMS algorithm . . . . . . . . . . . . . . . . . . . . . . . . . . 9\n1.2 The normal equations . . . . . . . . . . . . . . . . . . . . . . . 13\n1.2.1 Matrix derivatives . . . . . . . . . . . . . . . . . . . . . 13\n1.2.2 Least squares revisited . . . . . . . . . . . . . . . . . . 14\n1.3 Probabilistic interpretation . . . . . . . . . . . . . . . . . . . . 15\n1.4 Locally weighted linear regression (optional reading) . . . . . . 17\n2 Classiﬁcation and logistic regression 20\n2.1 Logistic regression . . . . . . . . . . . . . . . . . . . . . . . . 20\n2.2 Digression: the perceptron learning algorithn . . . . . . . . . . 23\n2.3 Another algorithm for maximizing ℓ(θ) . . . . . . . . . . . . . 24\n3 Generalized linear models 26\n3.1 The exponential family . . . . . . . . . . . . . . . . . . . . . . 26\n3.2 Constructing GLMs . . . . . . . . . . . . . . . . . . . . . . . . 28\n3.2.1 Ordinary least squares .

In [23]:
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo
import lark


In [24]:
metadata_field_info = [
    AttributeInfo(
        name="source",
        description="The source of the page within the lecture notes. It should always be `https://cs229.stanford.edu/lectures-spring2022/main_notes.pdf`",
        type="string",
    ),
    AttributeInfo(
        name="page",
        description="The page from the lecture",
        type="integer",
    ),
]

In [25]:
llm = OpenAI(model='gpt-3.5-turbo-instruct', temperature=0, api_key = api_key)
document_content_description = "Lecture notes"
retriever = SelfQueryRetriever.from_llm(llm, vectordb, document_content_description, metadata_field_info, verbose = True)

C:\Users\harsh\anaconda3.1\envs\pracopenai\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [28]:
self_query_retrieved_docs = retriever.invoke(query)
print(len(self_query_retrieved_docs))

2


In [31]:
self_query_retrieved_docs
for doc in self_query_retrieved_docs:
    print(f'The document is retrieved from the page: {doc.metadata}')

The document is retrieved from the page: {'page': 20, 'source': 'https://cs229.stanford.edu/lectures-spring2022/main_notes.pdf'}
The document is retrieved from the page: {'page': 20, 'source': 'https://cs229.stanford.edu/lectures-spring2022/main_notes.pdf'}


In [32]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [36]:
compressor = LLMChainExtractor.from_llm(llm)
compressed_retriver = ContextualCompressionRetriever(base_compressor = compressor, base_retriever = vectordb.as_retriever())

In [46]:
query = 'What is regression?'
vectordb.as_retriever().invoke(query)

[Document(page_content='Chapter 1\nLinear regression\nTo make our housing example more interesting, let’s consider a slightly richer\ndataset in which we also know the number of bedrooms in each house:\nLiving area (feet2)#bedrooms Price (1000 $s)\n2104 3 400\n1600 3 330\n2400 3 369\n1416 2 232\n3000 4 540\n.........\nHere, thex’s are two-dimensional vectors in R2. For instance, x(i)\n1is the\nliving area of the i-th house in the training set, and x(i)\n2is its number of\nbedrooms. (In general, when designing a learning problem, it will be up to\nyou to decide what features to choose, so if you are out in Portland gathering\nhousing data, you might also decide to include other features such as whether\neach house has a ﬁreplace, the number of bathrooms, and so on. We’ll say\nmore about feature selection later, but for now let’s take the features as\ngiven.)\nTo perform supervised learning, we must decide how we’re going to rep-\nresent functions/hypotheses hin a computer. As an initial

In [47]:
compressed_retriver.invoke(query)

[Document(page_content='- To perform supervised learning, we must decide how we’re going to rep-\nresent functions/hypotheses hin a computer. As an initial choice, let’s say\nwe decide to approximate yas a linear function of x:\nhθ(x) =θ0+θ1x1+θ2x2\nHere, theθi’s are the parameters (also called weights ) parameterizing the\nspace of linear functions mapping from XtoY.', metadata={'page': 8, 'source': 'https://cs229.stanford.edu/lectures-spring2022/main_notes.pdf'}),
 Document(page_content='- To perform supervised learning, we must decide how we’re going to rep-\nresent functions/hypotheses hin a computer. As an initial choice, let’s say\nwe decide to approximate yas a linear function of x:\nhθ(x) =θ0+θ1x1+θ2x2\nHere, theθi’s are the parameters (also called weights ) parameterizing the\nspace of linear functions mapping from XtoY.', metadata={'page': 8, 'source': 'https://cs229.stanford.edu/lectures-spring2022/main_notes.pdf'})]

In [48]:
compressed_retriever_mmr = ContextualCompressionRetriever(base_compressor = compressor, base_retriever = vectordb.as_retriever(search_type='mmr'))
compressed_retriever_mmr.invoke(query)

[Document(page_content='- To perform supervised learning, we must decide how we’re going to rep-\nresent functions/hypotheses hin a computer. As an initial choice, let’s say\nwe decide to approximate yas a linear function of x:\nhθ(x) =θ0+θ1x1+θ2x2\nHere, theθi’s are the parameters (also called weights ) parameterizing the\nspace of linear functions mapping from XtoY.', metadata={'page': 8, 'source': 'https://cs229.stanford.edu/lectures-spring2022/main_notes.pdf'}),
 Document(page_content='- "This chapter discusses tools to analyze and understand the generalization of machine learning models"\n- "Recall that for supervised learning problems, given a training dataset{(x(i),y(i))}n i=1, we typically learn a model hθby minimizing a loss/cost function J(θ)"\n- "The most important evaluation metric of a model is the loss on unseen test examples"\n- "Formally, we sample a test example ( x,y) from the so-called test distribution D, and measure the model’s error on it, by, e.g., the mean squar

In [49]:
from langchain.retrievers import SVMRetriever
from langchain.retrievers import TFIDFRetriever
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [52]:
svm_retriever = SVMRetriever.from_documents(split_pages,embedding)
tfidf_retriever = TFIDFRetriever.from_documents(split_pages)

In [53]:
svm_retriever.invoke(query)

[Document(page_content='Part III\nGeneralization and\nregularization\n102', metadata={'source': 'https://cs229.stanford.edu/lectures-spring2022/main_notes.pdf', 'page': 102}),
 Document(page_content='Chapter 1\nLinear regression\nTo make our housing example more interesting, let’s consider a slightly richer\ndataset in which we also know the number of bedrooms in each house:\nLiving area (feet2)#bedrooms Price (1000 $s)\n2104 3 400\n1600 3 330\n2400 3 369\n1416 2 232\n3000 4 540\n.........\nHere, thex’s are two-dimensional vectors in R2. For instance, x(i)\n1is the\nliving area of the i-th house in the training set, and x(i)\n2is its number of\nbedrooms. (In general, when designing a learning problem, it will be up to\nyou to decide what features to choose, so if you are out in Portland gathering\nhousing data, you might also decide to include other features such as whether\neach house has a ﬁreplace, the number of bathrooms, and so on. We’ll say\nmore about feature selection later, bu

In [54]:
tfidf_retriever.invoke(query)

[Document(page_content='Contents\nI Supervised learning 5\n1 Linear regression 8\n1.1 LMS algorithm . . . . . . . . . . . . . . . . . . . . . . . . . . 9\n1.2 The normal equations . . . . . . . . . . . . . . . . . . . . . . . 13\n1.2.1 Matrix derivatives . . . . . . . . . . . . . . . . . . . . . 13\n1.2.2 Least squares revisited . . . . . . . . . . . . . . . . . . 14\n1.3 Probabilistic interpretation . . . . . . . . . . . . . . . . . . . . 15\n1.4 Locally weighted linear regression (optional reading) . . . . . . 17\n2 Classiﬁcation and logistic regression 20\n2.1 Logistic regression . . . . . . . . . . . . . . . . . . . . . . . . 20\n2.2 Digression: the perceptron learning algorithn . . . . . . . . . . 23\n2.3 Another algorithm for maximizing ℓ(θ) . . . . . . . . . . . . . 24\n3 Generalized linear models 26\n3.1 The exponential family . . . . . . . . . . . . . . . . . . . . . . 26\n3.2 Constructing GLMs . . . . . . . . . . . . . . . . . . . . . . . . 28\n3.2.1 Ordinary least squares . 